In [5]:
!pip install geopandas shapely folium scikit-learn


In [6]:
from google.colab import files
uploaded = files.upload()


Saving grid_cells.geojson to grid_cells (2).geojson
Saving study_area_boundary.geojson to study_area_boundary (2).geojson
Saving wildfire_samples.csv to wildfire_samples (2).csv


In [7]:
import pandas as pd
import geopandas as gpd
import folium

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
# Tabular training data
wf = pd.read_csv("/content/wildfire_samples.csv")
print("Wildfire samples:")
print(wf.head())
print(wf.describe())

# Study area polygon
study = gpd.read_file("/content/study_area_boundary.geojson")
print("\nStudy area boundary:")
display(study)

# Grid cells for predictions
grid = gpd.read_file("/content/grid_cells.geojson")
print("\nGrid cells:")
print(grid.head())


Wildfire samples:
     elevation  vegetation_index  temperature  distance_to_road  burned
0  2083.172820          0.637074    34.645115          6.374689       1
1  1032.649267          0.219374    24.411505         15.088322       0
2  1147.037641          0.779473    22.518410         14.151536       0
3  1811.552320          0.831793    29.059649         19.355724       1
4  1314.573516          0.207274    21.071463          4.066735       0
         elevation  vegetation_index  temperature  distance_to_road  \
count   300.000000        300.000000   300.000000        300.000000   
mean   1221.938154          0.483796    27.276062         10.427302   
std     589.398536          0.235115     7.254353          5.808435   
min     201.366993          0.104156    15.032029          0.013519   
25%     715.158511          0.271108    21.141430          5.622119   
50%    1273.296159          0.484600    26.812384         10.417182   
75%    1720.115569          0.667710    33.808888    

,region_name,geometry
0,SampleRegion_Mountains,"POLYGON ((-120.8 38.5, -119.8 38.5, -119.8 39...."



Grid cells:
  cell_id                                           geometry
0      G1  POLYGON ((-120.8 38.5, -120.6 38.5, -120.6 38....
1      G2  POLYGON ((-120.6 38.5, -120.4 38.5, -120.4 38....
2      G3  POLYGON ((-120.4 38.5, -120.2 38.5, -120.2 38....
3      G4  POLYGON ((-120.2 38.5, -120 38.5, -120 38.64, ...
4      G5  POLYGON ((-120 38.5, -119.8 38.5, -119.8 38.64...


In [22]:
# Features (x) and target (y)
features_cols = ["elevation", "vegetation_index", "temperature", "distance_to_road"]
x = wf[features_cols]
y = wf["burned"]

# Train/test Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [16]:
# Random forest classifier
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
rf.fit(x_train, y_train)
print("Training done.")
print("Accuracy on training set: ", rf.score(x_train, y_train))

Training done.
Accuracy on training set:  1.0


In [17]:
# Evaluate the model

y_pred = rf.predict(x_test)
print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification report:")
print(classification_report(y_test, y_pred))


Confusion matrix:
[[64  0]
 [ 1 10]]

Classification report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        64
           1       1.00      0.91      0.95        11

    accuracy                           0.99        75
   macro avg       0.99      0.95      0.97        75
weighted avg       0.99      0.99      0.99        75



In [18]:
import numpy as np

# copy grid so that we do not modify the original
grid_feat = grid.copy()

np.random.seed(0)

grid_feat["elevation"] = np.random.uniform(300, 2000, len(grid_feat))
grid_feat["vegetation_index"] = np.random.uniform(0.2, 0.9, len(grid_feat))
grid_feat["temperature"] = np.random.uniform(20, 38, len(grid_feat))
grid_feat["distance_to_road"] = np.random.uniform(0, 15, len(grid_feat))

grid_feat[["cell_id", "elevation", "vegetation_index", "temperature", "distance_to_road"]].head()

,cell_id,elevation,vegetation_index,temperature,distance_to_road
0,G1,1232.982957,0.647945,30.263542,0.587817
1,G2,1515.821923,0.300347,27.894827,4.242104
2,G3,1324.697739,0.861268,37.790729,1.802948
3,G4,1226.301411,0.565294,21.836807,4.442103
4,G5,1020.213159,0.490263,23.759782,1.780916


In [23]:
X_grid = grid_feat[features_cols]

# Predict probability of "burned = 1"
grid_feat["risk_prob"] = rf.predict_proba(X_grid)[:, 1]

# Also create a simple risk category
def risk_level(p):
    if p >= 0.7:
        return "High"
    elif p >= 0.4:
        return "Medium"
    else:
        return "Low"

grid_feat["risk_level"] = grid_feat["risk_prob"].apply(risk_level)

grid_feat[["cell_id", "risk_prob", "risk_level"]].head()

,cell_id,risk_prob,risk_level
0,G1,0.20,Low
1,G2,0.00,Low
2,G3,0.36,Low
3,G4,0.00,Low
4,G5,0.00,Low


In [24]:
# Reproject to WGS84 if needed (should already be EPSG:4326)
grid_wgs = grid_feat.to_crs(epsg=4326)
study_wgs = study.to_crs(epsg=4326)

# Get map center from study area centroid
center = [
    study_wgs.geometry.centroid.y.mean(),
    study_wgs.geometry.centroid.x.mean()
]

m = folium.Map(location=center, zoom_start=8)

# Add study boundary outline
folium.GeoJson(
    study_wgs,
    name="Study Area",
    style_function=lambda x: {"fill": False, "color": "black", "weight": 2}
).add_to(m)

# Color by risk level
def style_function(feature):
    level = feature["properties"]["risk_level"]
    if level == "High":
        fill_color = "#d73027"  # red
    elif level == "Medium":
        fill_color = "#fee08b"  # yellow
    else:
        fill_color = "#1a9850"  # green
    return {
        "fillColor": fill_color,
        "color": "gray",
        "weight": 1,
        "fillOpacity": 0.6,
    }

folium.GeoJson(
    grid_wgs,
    name="Wildfire Risk",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["cell_id", "risk_level", "risk_prob"],
        aliases=["Cell ID", "Risk Level", "Risk Probability"],
        localize=True
    )
).add_to(m)

folium.LayerControl().add_to(m)

m


/tmp/ipython-input-1653533203.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  study_wgs.geometry.centroid.y.mean(),
/tmp/ipython-input-1653533203.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  study_wgs.geometry.centroid.x.mean()


In [25]:
# Save to GeoJSON
out_path = "grid_cells_with_risk.geojson"
grid_wgs.to_file(out_path, driver="GeoJSON")
out_path


'grid_cells_with_risk.geojson'